In [11]:
from datetime import datetime
import json
import os
import re

from lxml import etree
import requests

import utils


NameError: name 'BASE_URL' is not defined

In [3]:
def parseHTMLByXPath(content):
    ''' 使用 xpath 解析 HTML, 提取榜单信息

    Args:
        content: str, 待解析的 HTML 字符串
    Returns:
        榜单信息的字典 字典
    '''
    html = etree.HTML(content)

    titles = html.xpath(
        '//tr[position()>1]/td[@class="td-02"]/a[not(contains(@href, "javascript:void(0);"))]/text()')
    hrefs = html.xpath(
        '//tr[position()>1]/td[@class="td-02"]/a[not(contains(@href, "javascript:void(0);"))]/@href')
    hots = html.xpath(
        '//tr[position()>1]/td[@class="td-02"]/a[not(contains(@href, "javascript:void(0);"))]/../span/text()')
    titles = [title.strip() for title in titles]
    hrefs = [BASE_URL + href.strip() for href in hrefs]
    hots = [int(hot.strip().split(' ')[-1])
            for hot in hots]  # 该处除了热度还会返回大致分类，形如 `剧集 53412536`，前为分类，后为热度

    correntRank = {}
    for i, title in enumerate(titles):
        correntRank[title] = {'href': hrefs[i], 'hot': hots[i]}

    return correntRank


In [8]:
def getHTML(url):
    ''' 获取网页 HTML 返回字符串

    Args:
        url: str, 网页网址
    Returns:
        HTML 字符串
    '''
    # Cookie 有效期至2023-02-10
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36',
        'Cookie': 'SUB=_2AkMVWDYUf8NxqwJRmP0Sz2_hZYt2zw_EieKjBMfPJRMxHRl-yj9jqkBStRB6PtgY-38i0AF7nDAv8HdY1ZwT3Rv8B5e5; SUBP=0033WrSXqPxfM72-Ws9jqgMF55529P9D9WFencmWZyNhNlrzI6f0SiqP'
    }
    response = requests.get(url, headers=headers)
    if response.encoding == 'ISO-8859-1':
        response.encoding = response.apparent_encoding if response.apparent_encoding != 'ISO-8859-1' else 'utf-8'
    return response.text

In [9]:
content = getHTML("https://s.weibo.com/top/summary")

In [12]:
content

'<!DOCTYPE html>\n<html>\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge"/>\n    <meta name="renderer" content="webkit"/>\n    <meta name="viewport" content="initial-scale=1,minimum-scale=1"/>\n    <title>微博搜索-热搜榜</title>\n    <link rel="stylesheet" href="//js.t.sinajs.cn/t4/apps/searchpc/js/pcNew/top-nav/weibo-top-nav.css?version=202212011227">\n    <!-- <link rel="stylesheet" href="//js.t.sinajs.cn/t4/apps/searchpc/js/pcNew/wooui/wooui.css"> -->\n\n    <link href="//img.t.sinajs.cn/t4/appstyle/searchpc/css/new_pc/css/video.css?version=202212011227" rel="stylesheet"/>\n    <link href="//img.t.sinajs.cn/t4/appstyle/searchpc/css/new_pc/css/global.css?version=202212011227" rel="stylesheet" />\n    <link href="//img.t.sinajs.cn/t4/appstyle/searchpc/css/new_pc/css/module.css?version=202212011227" rel="stylesheet" />\n    <link href="//img.t.sinajs.cn/t4/appstyle/searchpc/css/new_pc/css/page.css?version=202212011227" rel="stylesheet" />\n    <lin

In [15]:
correntRank = parseHTMLByXPath(content)

In [14]:
BASE_URL = 'https://s.weibo.com'

In [16]:
correntRank

{'短发新娘不穿婚纱被误认成伴郎': {'href': 'https://s.weibo.com/weibo?q=%23%E7%9F%AD%E5%8F%91%E6%96%B0%E5%A8%98%E4%B8%8D%E7%A9%BF%E5%A9%9A%E7%BA%B1%E8%A2%AB%E8%AF%AF%E8%AE%A4%E6%88%90%E4%BC%B4%E9%83%8E%23&t=31&band_rank=1&Refer=top',
  'hot': 1458640},
 '灰太狼的羊称家暴出轨均为虚构': {'href': 'https://s.weibo.com/weibo?q=%23%E7%81%B0%E5%A4%AA%E7%8B%BC%E7%9A%84%E7%BE%8A%E7%A7%B0%E5%AE%B6%E6%9A%B4%E5%87%BA%E8%BD%A8%E5%9D%87%E4%B8%BA%E8%99%9A%E6%9E%84%23&t=31&band_rank=2&Refer=top',
  'hot': 1022598},
 '新冠康复如何做到清淡饮食': {'href': 'https://s.weibo.com/weibo?q=%23%E6%96%B0%E5%86%A0%E5%BA%B7%E5%A4%8D%E5%A6%82%E4%BD%95%E5%81%9A%E5%88%B0%E6%B8%85%E6%B7%A1%E9%A5%AE%E9%A3%9F%23&t=31&band_rank=3&Refer=top',
  'hot': 749894},
 '上海严公子': {'href': 'https://s.weibo.com/weibo?q=%23%E4%B8%8A%E6%B5%B7%E4%B8%A5%E5%85%AC%E5%AD%90%23&t=31&band_rank=4&Refer=top',
  'hot': 730438},
 '权志龙姐姐开北京首店': {'href': 'https://s.weibo.com/weibo?q=%23%E6%9D%83%E5%BF%97%E9%BE%99%E5%A7%90%E5%A7%90%E5%BC%80%E5%8C%97%E4%BA%AC%E9%A6%96%E5%BA%97%23&t=31&band_

In [20]:

def updateJSON(correntRank):
    ''' 更新当天的 JSON 文件

    Args:
        correntRank: dict, 最新的榜单信息
    Returns:
        与当天历史榜单对比去重, 排序后的榜单信息字典
    '''
    filename = datetime.today().strftime('%Y%m%d') + '.json'
    filename = os.path.join(JSON_DIR, filename)

    # 文件不存在则创建
    if not os.path.exists(filename):
        utils.save(filename, {})

    historyRank = json.loads(utils.load(filename))
    for k, v in correntRank.items():
        # 若当前榜单和历史榜单有重复的，取热度数值(名称后面的数值)更大的一个
        if k in historyRank:
            historyRank[k]['hot'] = max(
                historyRank[k]['hot'], correntRank[k]['hot'])
        # 若没有，则添加
        else:
            historyRank[k] = v

    # 将榜单按 hot 值排序
    rank = {k: v for k, v in sorted(
        historyRank.items(), key=lambda item: item[1]['hot'], reverse=True)}

    # 更新当天榜单 json 文件
    utils.save(filename, rank)
    return rank
JSON_DIR = './raw'
rankJSON = updateJSON(correntRank)

In [21]:
rankJSON 

{'权志龙姐姐开北京首店': {'href': 'https://s.weibo.com/weibo?q=%23%E6%9D%83%E5%BF%97%E9%BE%99%E5%A7%90%E5%A7%90%E5%BC%80%E5%8C%97%E4%BA%AC%E9%A6%96%E5%BA%97%23&t=31&band_rank=1&Refer=top',
  'hot': 2001437},
 '一家4口三亚溺水全部遇难': {'href': 'https://s.weibo.com/weibo?q=%23%E4%B8%80%E5%AE%B64%E5%8F%A3%E4%B8%89%E4%BA%9A%E6%BA%BA%E6%B0%B4%E5%85%A8%E9%83%A8%E9%81%87%E9%9A%BE%23&t=31&band_rank=2&Refer=top',
  'hot': 1789683},
 '短发新娘不穿婚纱被误认成伴郎': {'href': 'https://s.weibo.com/weibo?q=%23%E7%9F%AD%E5%8F%91%E6%96%B0%E5%A8%98%E4%B8%8D%E7%A9%BF%E5%A9%9A%E7%BA%B1%E8%A2%AB%E8%AF%AF%E8%AE%A4%E6%88%90%E4%BC%B4%E9%83%8E%23&t=31&band_rank=1&Refer=top',
  'hot': 1497125},
 '新冠康复如何做到清淡饮食': {'href': 'https://s.weibo.com/weibo?q=%23%E6%96%B0%E5%86%A0%E5%BA%B7%E5%A4%8D%E5%A6%82%E4%BD%95%E5%81%9A%E5%88%B0%E6%B8%85%E6%B7%A1%E9%A5%AE%E9%A3%9F%23&t=31&band_rank=3&Refer=top',
  'hot': 1492124},
 '电费1个月3481元竟有电费刺客': {'href': 'https://s.weibo.com/weibo?q=%23%E7%94%B5%E8%B4%B91%E4%B8%AA%E6%9C%883481%E5%85%83%E7%AB%9F%E6%9C%89%E7%94

In [6]:
kkk="4859348037930699"

In [7]:
len(kkk)

16